In [12]:
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Bidirectional
from keras.layers import Attention
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from google.colab import drive
drive.mount('/content/drive')
# from scipy.signal import argrelextrema
# from statsmodels.nonparametric.kernel_regression import KernelReg
def find_MaxtimeStep(df):
    x = df.loc[df['TREND'].notnull()].index.tolist()
    max_distance =0 
    for i in range (0,len(x)-1):
        if x[i+1]-x[i] >max_distance:
            max_distance = x[i+1]-x[i]
    return max_distance
def get_fixedSizeCandle(candles,md):
    candles_array = np.array(candles)
    Fixed_MaxSizeCandles = np.zeros((md,candles_array.shape[1]),dtype=float)
    Fixed_MaxSizeCandles[md-candles_array.shape[0]:,:candles_array.shape[1]] =candles_array
    return Fixed_MaxSizeCandles
def PrepareData(selecteddata):
    excel_data = pd.read_excel('/content/drive/MyDrive/Dataset/New_Dataset.xlsx')
    df = pd.DataFrame(excel_data, columns=['DATE', 'TIME', 'OPEN','HIGH','LOW','CLOSE','TREND'])
    Max_TimeStep = find_MaxtimeStep(df)
    x=list()
    y=list()
    i=0
    candles = list()
    while i<len(df):
        candle =list()
        candle = np.zeros(4)#0:Open,1:high,2:low,3:close,4:diff(O-C),5:diff(h-l),6:ma(o),7:ma(h),8:ma(l),9:ma(c)
        if not(pd.isnull(df.at[i,'TREND'])):
            y.append(df.at[i,'TREND'])            
            if len(candles)>0:
              # MV= pd.DataFrame(candles).rolling(5, min_periods=1).mean().values              
              # for k in range(0,len(candles)):
              #   for m in range(0,4):
              #     candles[k][m+6] = MV[k,m]
              x.append(get_fixedSizeCandle(candles,Max_TimeStep))
            candles = list()
        else:
          for j in range (0,4):
            candle[j] = df.iloc[i,j+2] -df.iloc[i-1,j+2]
            # candle[4] = candle[0]-candle[3]
            # candle[5] = candle[1]-candle[2]
            #candle[j] = df.iloc[i,j+2]
        candles.append(candle)
        i = i+1
    x.append(get_fixedSizeCandle(candles,Max_TimeStep))
    return np.array(x),np.array(y),Max_TimeStep

def Create_Model(num_timesteps,num_features):
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, input_shape=(num_timesteps, num_features)))
    model.add(Bidirectional(LSTM(32)))
    model.add(Dense(16))
    model.add(Dense(5,activation= 'softmax'))
    return model

def Show_history_plot(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
def Create_Train_Test(x,y,test_Persent):
    scaler = StandardScaler()
    for i in range(0,x.shape[0]):
        x[i]= scaler.fit_transform(x[i])
    #x = scaler.fit_transform(x.reshape(-1, x.shape[-1])).reshape(x.shape)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(y)
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
        
    X_Train = x[:int((100-test_Persent)*x.shape[0]/100),:,:]
    X_Test = x[(X_Train.shape[0]):,:,:]
    Y_Train = onehot_encoded[:X_Train.shape[0],:]
    Y_Test = onehot_encoded[X_Train.shape[0]:,:]
    return X_Train,X_Test,Y_Train,Y_Test

#set parameters
test_Persent = 15
Max_TimeStep=0
acc_per_fold = []
loss_per_fold = []
x,y,Max_TimeStep = PrepareData(4)
X_Train,X_Test,Y_Train,Y_Test = Create_Train_Test(x,y,test_Persent)
scaler = StandardScaler()
for i in range(0,x.shape[0]):
    x[i]= scaler.fit_transform(x[i])
#x = scaler.fit_transform(x.reshape(-1, x.shape[-1])).reshape(x.shape)

fold_no=1
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
for train, test in kfold.split(x, y):
  label_encoder = LabelEncoder()
  integer_encoded = label_encoder.fit_transform(y[train])
  onehot_encoder = OneHotEncoder(sparse=False)
  integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
  Y_Train = onehot_encoder.fit_transform(integer_encoded)

  integer_encoded = label_encoder.fit_transform(y[test])
  onehot_encoder = OneHotEncoder(sparse=False)
  integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
  Y_Test = onehot_encoder.fit_transform(integer_encoded)

  X_Train, X_Test = x[train], x[test]
  #Y_Train, Y_Test = y[train], y[test]

  model = Create_Model(Max_TimeStep,X_Train.shape[2])
  model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
  history= model.fit(X_Train, Y_Train, epochs=30,validation_split=0.1,  verbose=0)
   #Show_history_plot(history)
  #testPredict = model.predict(X_Test)
  scores =model.evaluate(X_Test,Y_Test,verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])
  fold_no = fold_no + 1

print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')
# classes = ['D','S','U']
# result = list()
# for i in range(0,len(testPredict)):
#   result.append([classes[np.argmax(Y_Test[i])],classes[np.argmax(testPredict[i])]])
# print(result)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Score for fold 1: loss of 0.2632380425930023; accuracy of 95.99999785423279%
Score for fold 2: loss of 0.11648133397102356; accuracy of 95.99999785423279%
Score for fold 3: loss of 0.41434624791145325; accuracy of 92.00000166893005%
Score for fold 4: loss of 0.1707179844379425; accuracy of 95.99999785423279%
Score for fold 5: loss of 0.14679504930973053; accuracy of 95.99999785423279%
Score for fold 6: loss of 0.6622216105461121; accuracy of 83.99999737739563%
Score for fold 7: loss of 0.31401997804641724; accuracy of 92.00000166893005%
Score for fold 8: loss of 0.14988936483860016; accuracy of 95.99999785423279%
Score for fold 9: loss of 0.45227551460266113; accuracy of 80.0000011920929%
Score for fold 10: loss of 0.11979996412992477; accuracy of 95.83333134651184%
------------------------------------------------------------------------
Score per fold
------